# Flow Sort I

This module shows an example of how to use the "Flow Sort I" module.

In [2]:
%config Completer.use_jedi = False

## Definition of inputs and problem formalization

In [4]:
import pandas as pd
from modular_parts.preference import compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows, calculate_net_outranking_flows
from modular_parts.sorting import calculate_flowsortI_sorted_alternatives
from core.enums import Direction, GeneralCriterion, FlowType, CompareProfiles


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)
profiles_performances = pd.DataFrame([[12, 80, 16],
                                      [15, 68, 18],
                                      [18, 55, 20]], index=profiles, columns=criteria)

central_categories = [f"C{i}" for i in range(1, 4)]
boundary_categories = [f"C{i}" for i in range(1, 5)]
limiting_categories = [f"C{i}" for i in range(1, 3)]

## Required data

### Required preferences

In [5]:
aggregated_altenatives_vs_profiles_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights,
                                                                    profiles_performances)

aggregated_profiles_vs_profiles_preferences, _ = compute_preference_indices(profiles_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)

In [6]:
aggregated_altenatives_vs_profiles_preferences[0]

,p1,p2,p3
a1,0.8,0.50,0.5
a2,0.0,0.00,0.0
a3,0.5,0.50,0.0
a4,1.0,0.96,0.0
a5,1.0,0.92,0.5
a6,0.5,0.50,0.5
a7,0.7,0.54,0.5


In [7]:
aggregated_altenatives_vs_profiles_preferences[1]

,a1,a2,a3,a4,a5,a6,a7
p1,0.0,1.0,0.00,0.0,0.0,0.5,0.00
p2,0.2,1.0,0.38,0.0,0.0,0.5,0.30
p3,0.5,1.0,0.50,0.0,0.0,0.5,0.38


In [8]:
aggregated_profiles_vs_profiles_preferences

,p1,p2,p3
p1,0.0,0.0,0.0
p2,1.0,0.0,0.0
p3,1.0,1.0,0.0


### Required flows

In [9]:
alternatives_vs_profiles_basic_flows = calculate_promethee_outranking_flows(aggregated_altenatives_vs_profiles_preferences,
                                                                            flow_type=FlowType.BASIC)

profiles_vs_profiles_basic_flows = calculate_promethee_outranking_flows(aggregated_profiles_vs_profiles_preferences,
                                                                            flow_type=FlowType.BASIC)


In [10]:
alternatives_vs_profiles_basic_flows

,positive,negative
a1,0.600000,0.233333
a2,0.000000,1.000000
a3,0.333333,0.293333
a4,0.653333,0.000000
a5,0.806667,0.000000
a6,0.500000,0.500000
a7,0.580000,0.226667


In [11]:
profiles_vs_profiles_basic_flows

,positive,negative
p1,0.0,1.0
p2,0.5,0.5
p3,1.0,0.0


## Usage of FlowSort I

### Central profiles

In [13]:
sorted_alternatives = calculate_flowsortI_sorted_alternatives(central_categories,
                                                              profiles_performances,
                                                              criteria_directions,
                                                              alternatives_vs_profiles_basic_flows,
                                                              profiles_vs_profiles_basic_flows,
                                                              CompareProfiles.CENTRAL_PROFILES)
sorted_alternatives

,worse,better
a1,C2,C3
a2,C1,C1
a3,C2,C2
a4,C2,C3
a5,C3,C3
a6,C2,C2
a7,C2,C3


### Boundary profiles

In [14]:
sorted_alternatives = calculate_flowsortI_sorted_alternatives(boundary_categories,
                                                              profiles_performances,
                                                              criteria_directions,
                                                              alternatives_vs_profiles_basic_flows,
                                                              profiles_vs_profiles_basic_flows,
                                                              CompareProfiles.BOUNDARY_PROFILES)
sorted_alternatives

,worse,better
a1,C3,C3
a2,C1,C2
a3,C2,C3
a4,C3,C3
a5,C3,C3
a6,C2,C3
a7,C3,C3


### Limiting profiles

In [15]:
sorted_alternatives = calculate_flowsortI_sorted_alternatives(limiting_categories,
                                                              profiles_performances,
                                                              criteria_directions,
                                                              alternatives_vs_profiles_basic_flows,
                                                              profiles_vs_profiles_basic_flows,
                                                              CompareProfiles.LIMITING_PROFILES)
sorted_alternatives

,worse,better
a1,C2,C2
a2,NaN,NaN
a3,C1,C2
a4,NaN,NaN
a5,NaN,NaN
a6,C1,C2
a7,C2,C2


NaN means that the alternative exceeds the classification (is below the worst profile or above the best profile).